In [72]:
import inspect
import multiprocessing as mp
import numpy as np
import os
import threading
import time


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from msbd.scraping import ottieni_contenuto_url

print(inspect.getsource(ottieni_contenuto_url))

def ottieni_contenuto_url(url, coda=None):
    id_processo = os.getpid()
    nome_processo = mp.current_process().name
    nome_thread = threading.current_thread().name
    
    print("[INIZIO]\nURL: {}\nID processo: {}\nNome processo: {}\nNome thread:"
        "{}\n".format(url, id_processo, nome_processo, nome_thread))

    try:
        risposta = requests.get(url)
        testo = BeautifulSoup(risposta.content, "lxml").get_text()
    except requests.exceptions.HTTPError as err:
        print("HTTP Error:", err)
    except requests.exceptions.ConnectionError as err:
        print("Connection Error:", err)
    except requests.exceptions.Timeout as err:
        print("Timeout Error:", err)
    except requests.exceptions.RequestException as err:
        print("Request Error",err)
        
    print("[FINE]\nURL: {}\nID processo: {}\nNome processo: {}\nNome thread:"
        "{}\n".format(url, id_processo, nome_processo, nome_thread))
    
    if coda is not None:
        coda.put(testo)


In [61]:
contenuto = ottieni_contenuto_url("https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0")

[INIZIO]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[FINE]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread



In [70]:
N = 5

righe_non_vuote = [c for c in contenuto.split("\n") if c.strip()]

print("Prime {} righe non vuote:".format(N) + ("\n{}" * N).format(*righe_non_vuote[:N]))

Prime 5 righe non vuote:
Didattica - Università degli Studi di Padova
Vai alla navigazione
Unipd.it
Rubrica
IT


In [94]:
from msbd.scraping import ottieni_contenuto_urls_sequenziale

print(inspect.getsource(ottieni_contenuto_urls_sequenziale))

def ottieni_contenuto_urls_sequenziale(urls):
    # ============== YOUR CODE HERE ==============
    contenuti = [ottieni_contenuto_url(url) for url in URLS]
    # ============================================

    return contenuti



### Esercizio

Utilizzando *list comprehension* (o un ciclo *for*) e la funzione `ottieni_contenuto_url()` per completare la funzione `ottieni_contenuto_urls_sequenziale()` in `msbd/modello_lineare/scraping.py`.

In [93]:
URLS = [
    "https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0",
    "https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python",
    "https://docs.python.org/3.6/library/threading.html",
    "https://docs.python.org/3.6/library/multiprocessing.html",
]

In [97]:
inizio = time.time()

# ============== YOUR CODE HERE ==============
contenuti = ottieni_contenuto_urls_sequenziale(URLS)
# ============================================

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

[INIZIO]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[FINE]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[INIZIO]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[FINE]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[INIZIO]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[FINE]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12216
Nome processo: MainProcess
Nome thread: MainThread

[INIZIO]
URL: https://docs.python.org/3.6/library/multiprocessing.html
ID processo: 1221

# Programmazione concorrente: *multithreading*

In [105]:
from msbd.scraping import ottieni_contenuto_urls_threading

print(inspect.getsource(ottieni_contenuto_urls_threading))

def ottieni_contenuto_urls_threading(urls):
    coda = mp.Queue()

    threads = [threading.Thread(target=ottieni_contenuto_url, args=(url, coda))
        for url in urls]

    for t in threads:
        t.start()

    contenuti = [coda.get() for t in threads]

    for t in threads:
        t.join() # blocca il MainThread finché t non è completato

    return contenuti



In [117]:
inizio = time.time()

contenuti = ottieni_contenuto_urls_threading(URLS)

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

[INIZIO]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-42

[INIZIO]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-43

[INIZIO]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-44

[INIZIO]
URL: https://docs.python.org/3.6/library/multiprocessing.html
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-45

[FINE]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-42

[FINE]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12216
Nome processo: MainProcess
Nomethread: Thread-44

[FINE]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12216
Nome proce

### Esercizio

Descrivere cosa è cambiato rispetto all'esecuzione sequenziale.

In [128]:
%timeit -r 7 -n 1 ottieni_contenuto_urls_sequenziale(URLS, verboso=False)

1.95 s ± 82.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [129]:
%timeit -r 7 -n 1 ottieni_contenuto_urls_threading(URLS, verboso=False)

1.22 s ± 21.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Programmazione concorrente: *multiprocessing*

In [134]:
n_cpu = mp.cpu_count()

print("# CPU: {}".format(n_cpu))

# CPU: 4


In [135]:
from msbd.scraping import ottieni_contenuto_urls_multiprocessing

print(inspect.getsource(ottieni_contenuto_urls_multiprocessing))

def ottieni_contenuto_urls_multiprocessing(urls, verboso=True):
    # ============== YOUR CODE HERE ==============
    coda = mp.Queue()

    processi = [mp.Process(target=ottieni_contenuto_url, args=(url,),
        kwargs={"coda": coda, "verboso": verboso}) for url in urls]

    for p in processi:
        p.start()

    contenuti = [coda.get() for p in processi]

    for p in processi:
        p.join() # blocca il MainThread finché p non è completato
    # ============================================

    return contenuti



### Esercizio

Completare la funzione `ottieni_contenuto_urls_multiprocessing()` in `msbd/modello_lineare/scraping.py`.

> Suggerimento: imitare la funzione `ottieni_contenuto_urls_threading()` sostituendo alla classe `Thread` contenuta nella libreria `threading` la classe `Process` contenuta nella libreria `multiprocessing` (già importata in `scraping.py` come `mp`).

In [131]:
inizio = time.time()

contenuti = ottieni_contenuto_urls_multiprocessing(URLS)

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

[INIZIO]
URL: https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0
ID processo: 12656
Nome processo: Process-1
Nomethread: MainThread

[INIZIO]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12657
Nome processo: Process-2
Nomethread: MainThread
[INIZIO]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12658
Nome processo: Process-3
Nomethread: MainThread


[INIZIO]
URL: https://docs.python.org/3.6/library/multiprocessing.html
ID processo: 12659
Nome processo: Process-4
Nomethread: MainThread

[FINE]
URL: https://docs.python.org/3.6/library/threading.html
ID processo: 12658
Nome processo: Process-3
Nomethread: MainThread

[FINE]
URL: https://docs.python.org/3.6/library/multiprocessing.html
ID processo: 12659
Nome processo: Process-4
Nomethread: MainThread

[FINE]
URL: https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python
ID processo: 12657
Nome processo: Process-2
Nomethr

### Esercizio

1. Descrivere le differenze rispetto al risultato ottenuto con l'approccio sequenziale;
2. Descrivere le differenze rispetto al risultato ottenuto con l'approccio multi thread.

### Esercizio

Utilizzare il comando magico `%timeit` per calcolare il tempo di esecuzione di `ottieni_contenuto_urls_multiprocessing(URLS, verboso=False)`

In [132]:
%timeit -r 7 -n 1 ottieni_contenuto_urls_multiprocessing(URLS, verboso=False)

1.37 s ± 191 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [138]:
import math

In [173]:
def test_primalita(n):
    return ((n >= 2) & all(n % i for i in range(2, n)))

def conteggio_numeri_primi_in_un_insieme(insieme):
    return sum(test_primalita(n) for n in insieme)

In [175]:
%timeit conteggio_numeri_primi_in_un_insieme(np.random.randint(0, 100000, size=1000))

1.04 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [179]:
insiemi = [np.random.randint(0, 100000, size=1000) for _ in range(4)]

In [181]:
for insieme in insiemi:
    print(conteggio_numeri_primi_in_un_insieme(insieme))

112
80
100
115


In [208]:
insiemi = [np.random.randint(50000, 100000, size=1000) for _ in range(12)]

In [209]:
%%timeit -r 1

pool = mp.Pool(processes=4)
sum([pool.apply(conteggio_numeri_primi_in_un_insieme, args=(insieme,)) for insieme in insiemi])

28.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [210]:
import math

In [212]:
sum(math.factorial(i) for i in range(1, 6))

153

In [207]:
%timeit -r 1 sum([conteggio_numeri_primi_in_un_insieme(insieme) for insieme in insiemi])

5.68 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
def conta_numeri_primi_pool_apply(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum(risultati)

def conta_numeri_primi_pool_apply_async(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply_async(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum([p.get() for p in risultati])

In [ ]:
n = conta_numeri_primi_pool_apply(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
n = conta_numeri_primi_pool_apply_async(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread, dormi={0: 1})

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process, dormi={0: 1})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool(numeri)

In [ ]:
pool = mp.Pool(processes=4)

risultati = [pool.apply_async(conta_numeri_primi, args=(n, {}, False)) 
             for n in np.array_split(numeri, 100)] 
sum([p.get() for p in risultati])

In [ ]:
pool = mp.Pool(processes=4)
results = [pool.apply_async(cube, args=(x,)) for x in range(1,7)]
output = [p.get() for p in results]
print(output)